In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTForImageClassification, ViTConfig
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus.validators import ModuleValidator

In [15]:
# CIFAR-10 dataset transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit ViT input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Load pre-trained ViT model from HuggingFace Transformers
config = ViTConfig.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTForImageClassification(config)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Loss function
criterion = nn.CrossEntropyLoss()


In [ ]:
# Set up the Privacy Engine
privacy_engine = PrivacyEngine(accountant='rdp')

# Verify that the model is valid for differential privacy (checks batchnorm layers)

model = ModuleValidator.fix(model)
# Attach Privacy Engine to the model and optimizer
model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=1.1,  # Add noise for differential privacy
    max_grad_norm=1.0,  # Clip gradients to this norm
)




In [ ]:
def train_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs).logits
        loss = criterion(outputs, labels)

        # Backward pass with gradient clipping and adding noise
        loss.backward()
        optimizer.step()

        # Track the loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    return avg_loss, accuracy

# Training Loop
def train(model, train_loader, optimizer, criterion, epochs=10):
    for epoch in range(epochs):
        avg_loss, accuracy = train_epoch(model, train_loader, optimizer, criterion)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")
        
        # Privacy accounting step
        epsilon, best_alpha = privacy_engine.get_privacy_spent(1e-5)
        print(f"(ε = {epsilon:.2f}, δ = {1e-5})")
        
# Start the training process
train(model, train_loader, optimizer, criterion, epochs=10)
